In [1]:
import os
os.environ["HTTP_PROXY"] = "http://http.proxy.fmr.com:8000"
os.environ["HTTPS_PROXY"] = "http://http.proxy.fmr.com:8000"
os.environ["https_proxy"] = "http://http.proxy.fmr.com:8000"

import json
from datasets import load_metric
import copy
from research.constants import EvalField
from research.utils.hybridqa.hybridqa import hybridqa_metric_format_func

In [2]:
with open("outputs/new-paper-results/hybridqa/gpt-4-blendsql-deepseek-coder/predictions.json", "r") as f:
    new = json.load(f)
with open("outputs/paper-results/hybridqa/gpt-4-blendsql-plus-pp/predictions.json", "r") as f:
    old = json.load(f)
for idx, item in enumerate(old):
    old[idx][EvalField.GOLD_ANSWER] = item.pop("answer_text")
    old[idx][EvalField.PREDICTION] = item.pop("pred_text")
    old[idx][EvalField.UID] = item.pop("id")
    old[idx][EvalField.PRED_BLENDSQL] = item.pop("pred_sql")

In [16]:
def is_empty_blendsql_pred(p: list):
    if len(p) == 0 or (
        any(x in str(p[0]) for x in ["table", "passage", "text"])
        and any(x in str(p[0]) for x in ["not", "empty"])
    ) or str(p[0]).strip() == "":
        return True
    return False
m = load_metric(path="./metrics/hybridqa/")
metric_format_func = hybridqa_metric_format_func

/Users/a720504/miniconda3/envs/blendsql/lib/python3.9/site-packages/datasets/load.py:848: FutureWarning: The repository for hybridqa contains custom code which must be executed to correctly load the metric. You can inspect the repository content at ./metrics/hybridqa/hybridqa.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [17]:
print("HybridQA BlendSQL 3 rows results:")
for item in new:
    _item = copy.deepcopy(item)
    if item['solver'] == "blendsql":
        if is_empty_blendsql_pred(_item[EvalField.PREDICTION]):
            _item[EvalField.PREDICTION] = ""
        m.add(**hybridqa_metric_format_func(item))
    else:
        _item[EvalField.PREDICTION] = ""
        m.add(**hybridqa_metric_format_func(_item))
print(json.dumps(m.compute(), indent=4))

HybridQA BlendSQL 3 rows results:
['the text does not provide information on who finished in position 4 in the 2004 united states grand prix.']
['']
['']
[]
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['the table does not provide information on the registry of the ship.']
['']
['']
['']
[]
[]
['']
[]
['']
['']
['']
['']
[]
['']
['the table does not provide information on which university is known as the lychee kingdom.']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
[]
['']
['']
['']
['']
['']
['the table does not provide information on where the 2nd leg was held during the most recent championships.']
['']
[]
[]
['']
['']
['']
['']
['']
['']
['']
[]
['']
['the table does not provide information on how many albums the singer and song-writer has sold.']
[]
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
['']
[]
['']
['the table does not provide information on who developed the game that was a spinoff of the guitar hero series.']
[]
['']
['']
['']
['t

In [4]:
print("HybridQA BlendSQL New 3 rows results:")
for item in new:
    _item = copy.deepcopy(item)
    if item['solver'] == "blendsql":
        if is_empty_blendsql_pred(_item[EvalField.PREDICTION]):
             _item[EvalField.PREDICTION] = ""
        m.add(**hybridqa_metric_format_func(item))
    else:
        _item[EvalField.PREDICTION] = ""
        m.add(**hybridqa_metric_format_func(_item))
print(json.dumps(m.compute(), indent=4))

HybridQA BlendSQL New 3 rows results:
{
    "acc": 0.39959607616849396,
    "f1": 0.4721062063404212,
    "denotation_acc": 0.515868436237738
}


In [29]:
# Where did 'old' get right, and 'new' get wrong?
diff = []
for n, o in zip(new, old):
    if o["solver"] == "blendsql":
        m.add(**hybridqa_metric_format_func(o))
        o_correct = (m.compute()['denotation_acc'] == 1.0)
        m.add(**hybridqa_metric_format_func(n))
        n_correct = (m.compute()['denotation_acc'] == 1.0)
        o[EvalField.PRED_BLENDSQL] = rf'{o[EvalField.PRED_BLENDSQL]}'
        n[EvalField.PRED_BLENDSQL] = rf'{n[EvalField.PRED_BLENDSQL]}'
        if o_correct and (not n_correct):
            to_add = {
                "old": o,
                "new": n
            }
            diff.append(to_add)
with open("new_vs_old.json", "w") as f:
    json.dump(diff, f, indent=4)

In [31]:
print("ant\u00f3nio")

antónio


In [25]:
old_none = 0
new_none = 0
total_mismatched = 0
with open("new_errors.txt", "w") as f:
    for n, o in zip(new, old):
        if n["error"] != o["error"]:
            total_mismatched += 1
            if o["error"] is None:
                old_none += 1
                f.write(f"{o[EvalField.QUESTION]}\n")
                f.write(f"{o[EvalField.PRED_BLENDSQL]}\n")
                f.write(f"{n[EvalField.PRED_BLENDSQL]}\n")
                f.write("\n")
                f.write(f"{o['error']}\n")
                f.write(f"{n['error']}\n")
                f.write("------------------------------------\n\n")
            if n["error"] is None:
                new_none += 1


In [24]:
EvalField.PRED_BLENDSQL

'pred_blendsql'

In [5]:
total_mismatched

715

In [6]:
old_none

206

In [7]:
new_none

304

In [39]:
import re

In [87]:
q = """SELECT "original chapter" FROM w WHERE "name" = {{
    LLMQA(
        'Who elected Tijjani Muhammad-Bande of Nigeria on June 4, 2019?',
        (SELECT title, content FROM documents WHERE documents MATCH "hello-friend OR loop"),
        options='w::name'
    )
}}"""

In [88]:
for match in re.finditer(r'(?<=MATCH )(\'|\").*(\'|\")', q):
    fts5_q = match.group()
    q = re.sub(r'MATCH {}'.format(fts5_q), f'MATCH \'{re.sub(r"-", " ", fts5_q[1:-1])}\'', q)
    q = re.sub(r'MATCH {}'.format(fts5_q), f'MATCH \'{re.sub(r"[^0-9a-zA-Z ]", "", fts5_q[1:-1])}\'', q)


In [89]:
print(q)

SELECT "original chapter" FROM w WHERE "name" = {{
    LLMQA(
        'Who elected Tijjani Muhammad-Bande of Nigeria on June 4, 2019?',
        (SELECT title, content FROM documents WHERE documents MATCH 'hello friend OR loop'),
        options='w::name'
    )
}}


In [47]:
[i for i in re.finditer(r'(?<=MATCH )(\'|\").*(\'|\")', q)]

[<re.Match object; span=(204, 228), match="'Tijjani Muhammad-Bande'">]

In [1]:
blendsql = """
{{
    LLMQA(
        'What is the nickname of the gold medal winner in the men\'s heavyweight greco-roman wrestling event of the 1932 Summer Olympics?',
        (
            SELECT documents.title AS 'Athlete', documents.content FROM documents
            JOIN {{
                LLMJoin(
                    left_on='w::name',
                    right_on='documents::title'
                )
            }} WHERE w.medal = 'gold' AND w.sport = 'wrestling ( greco-roman )' AND w.event = 'men\'s heavyweight'
        )
    )
}}
"""
import re

In [6]:
# Fix escaped quotes
for match in [i for i in re.finditer(r'(?<=\=) ?((\'|\").*?(\'|\"))(\s|$)', blendsql)][::-1]:
    text = match.group(1)
    if text[0] == '"':
        blendsql = blendsql[:match.start()] + text[0] + re.sub('"', ' ', text[1:-1]) + text[-1] + blendsql[match.end():]
    elif text[0] == "'":
        blendsql = blendsql[:match.start()] + text[0] + re.sub("'", ' ', text[1:-1]) + text[-1] + blendsql[match.end():]
    # blendsql = blendsql[:match.start()] + " " + re.sub(r'\\.', ' ', text) + " " + blendsql[match.end():]

# Fix escaped strings in single quotes
for match in [i for i in re.finditer(r'(\'.*?\')(,)', blendsql)][::-1]:
    text = match.group(1)
    # print(text)
    if "'" in text[1:-1]:
        print(text)
        blendsql = re.sub(text[1:-1], text[1:-1].replace("'", " "), blendsql)

blendsql = blendsql.replace("`", "'")
blendsql = blendsql.replace("{{LLM(", "{{LLMMap(")
# Below fixes case where we miss a ')'
# SELECT MAX({{LLMMap('total penalties?', 'w::penalties (p+p+s+s)'}}) FROM w
blendsql = re.sub(r"((MAX|MIN)\(\{\{.*?)(\'}}\))", r"\1')}})", blendsql)
blendsql = re.sub("'}}", "')}}", blendsql)
# Handle mistakes like {{LLMMap('field goal percentage?'; 'w::field goal\xa0%')}}
blendsql = re.sub(r"(\'|\"); ", r"\1,", blendsql)
quotes_start_end = [i.start() for i in re.finditer(r"(\'|\")", blendsql)]
quotes_start_end_spans = list(zip(*(iter(quotes_start_end),) * 2))


# Fix common non-alphanumeric FTS5 mistakes
for match in re.finditer(r'(?<=MATCH )(\'|\").*(\'|\")', blendsql):
    fts5_q = match.group()
    blendsql = re.sub(r'MATCH {}'.format(fts5_q), f'MATCH \'{re.sub(r"-", " ", fts5_q[1:-1])}\'', blendsql)
    blendsql = re.sub(r'MATCH {}'.format(fts5_q), f'MATCH \'{re.sub(r"[^0-9a-zA-Z ]", "", fts5_q[1:-1])}\'', blendsql)


"'Was her body discovered in Stave Lake?'"

In [8]:
print(blendsql)

'coquitlam college'AND {{LLMMap('Was her body discovered in Stave Lake?'


In [9]:
for match in [i for i in re.finditer(r'(?<=\=) ?((\'|\").*?(\'|\"))(\s|$)', blendsql)][::-1]:
    text = match.group(1)
    print(text)


SELECT EXISTS (
    SELECT * FROM w0 WHERE "education" ='coquitlam college AND {{LLMMap( Was her body discovered in Stave Lake?', 'w0::body discovered')}} = TRUE
)



In [53]:
for match in [i for i in re.finditer(r'(\'.*?\')(,)', blendsql)][::-1]:
    text = match.group(1)
    if "'" in text[1:-1]:
        blendsql = re.sub(text, text.replace("'", ""), blendsql)

In [54]:
print(blendsql)


{{
    LLMQA(
        'What place did the winner of the gold medal in men's k1 200 metres canoeing at the 2013 Summer Universiade have in the 2016 Olympics?',
        (
            SELECT documents.title AS 'Athlete', documents.content FROM documents
            JOIN {{
                LLMJoin(
                    left_on='w::name',
                    right_on='documents::title'
                )
            }} WHERE w.medal = 'gold' AND w.sport = 'canoeing's friend'
        )
    )
}}



In [36]:
print(re.sub(r'\'', '', blendsql))


{{
    LLMQA(
        What place did the winner of the gold medal in mens k1 200 metres canoeing at the 2013 Summer Universiade have in the 2016 Olympics?,
        (
            SELECT documents.title AS Athlete, documents.content FROM documents
            JOIN {{
                LLMJoin(
                    left_on=w::name,
                    right_on=documents::title
                )
            }} WHERE w.medal = gold AND w.sport = canoeing
        )
    )
}}

